In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [3]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [6]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch  1, Batch   1 -Loss: 66539.5078 Validation Accuracy: 0.113281
Epoch  1, Batch   2 -Loss: 53620.0781 Validation Accuracy: 0.132812
Epoch  1, Batch   3 -Loss: 40865.7031 Validation Accuracy: 0.160156
Epoch  1, Batch   4 -Loss: 38801.0820 Validation Accuracy: 0.175781
Epoch  1, Batch   5 -Loss: 33324.8047 Validation Accuracy: 0.187500
Epoch  1, Batch   6 -Loss: 31088.0977 Validation Accuracy: 0.210938
Epoch  1, Batch   7 -Loss: 28536.3984 Validation Accuracy: 0.210938
Epoch  1, Batch   8 -Loss: 23968.7930 Validation Accuracy: 0.210938
Epoch  1, Batch   9 -Loss: 23770.1953 Validation Accuracy: 0.234375
Epoch  1, Batch  10 -Loss: 16676.7695 Validation Accuracy: 0.246094
Epoch  1, Batc

Epoch  1, Batch 112 -Loss:  3929.2227 Validation Accuracy: 0.710938
Epoch  1, Batch 113 -Loss:  4335.7402 Validation Accuracy: 0.707031
Epoch  1, Batch 114 -Loss:  4050.8936 Validation Accuracy: 0.710938
Epoch  1, Batch 115 -Loss:  3350.3135 Validation Accuracy: 0.710938
Epoch  1, Batch 116 -Loss:  3402.3591 Validation Accuracy: 0.710938
Epoch  1, Batch 117 -Loss:  4356.5869 Validation Accuracy: 0.714844
Epoch  1, Batch 118 -Loss:  3703.2637 Validation Accuracy: 0.714844
Epoch  1, Batch 119 -Loss:  2505.0552 Validation Accuracy: 0.722656
Epoch  1, Batch 120 -Loss:  4695.6143 Validation Accuracy: 0.714844
Epoch  1, Batch 121 -Loss:  3351.6279 Validation Accuracy: 0.718750
Epoch  1, Batch 122 -Loss:  3736.4585 Validation Accuracy: 0.730469
Epoch  1, Batch 123 -Loss:  4429.7866 Validation Accuracy: 0.722656
Epoch  1, Batch 124 -Loss:  4694.6660 Validation Accuracy: 0.730469
Epoch  1, Batch 125 -Loss:  3499.2783 Validation Accuracy: 0.722656
Epoch  1, Batch 126 -Loss:  3271.2839 Validation

Epoch  1, Batch 233 -Loss:  1871.6198 Validation Accuracy: 0.765625
Epoch  1, Batch 234 -Loss:  2647.4419 Validation Accuracy: 0.765625
Epoch  1, Batch 235 -Loss:  1851.3159 Validation Accuracy: 0.765625
Epoch  1, Batch 236 -Loss:  2691.7532 Validation Accuracy: 0.765625
Epoch  1, Batch 237 -Loss:  2036.6406 Validation Accuracy: 0.769531
Epoch  1, Batch 238 -Loss:  3613.7571 Validation Accuracy: 0.773438
Epoch  1, Batch 239 -Loss:  2717.5776 Validation Accuracy: 0.765625
Epoch  1, Batch 240 -Loss:  2583.8792 Validation Accuracy: 0.773438
Epoch  1, Batch 241 -Loss:  2721.1572 Validation Accuracy: 0.773438
Epoch  1, Batch 242 -Loss:  2250.9653 Validation Accuracy: 0.773438
Epoch  1, Batch 243 -Loss:  1575.2393 Validation Accuracy: 0.773438
Epoch  1, Batch 244 -Loss:  1642.6533 Validation Accuracy: 0.785156
Epoch  1, Batch 245 -Loss:  2347.6726 Validation Accuracy: 0.781250
Epoch  1, Batch 246 -Loss:  2283.8599 Validation Accuracy: 0.781250
Epoch  1, Batch 247 -Loss:  2405.4763 Validation

Epoch  1, Batch 354 -Loss:  1388.5281 Validation Accuracy: 0.796875
Epoch  1, Batch 355 -Loss:  2177.3242 Validation Accuracy: 0.792969
Epoch  1, Batch 356 -Loss:  1532.2463 Validation Accuracy: 0.800781
Epoch  1, Batch 357 -Loss:  1934.5635 Validation Accuracy: 0.796875
Epoch  1, Batch 358 -Loss:  1327.1244 Validation Accuracy: 0.789062
Epoch  1, Batch 359 -Loss:  1820.7925 Validation Accuracy: 0.792969
Epoch  1, Batch 360 -Loss:  2492.1187 Validation Accuracy: 0.792969
Epoch  1, Batch 361 -Loss:  2270.1162 Validation Accuracy: 0.785156
Epoch  1, Batch 362 -Loss:  1566.5532 Validation Accuracy: 0.789062
Epoch  1, Batch 363 -Loss:  1535.3005 Validation Accuracy: 0.800781
Epoch  1, Batch 364 -Loss:  1611.0657 Validation Accuracy: 0.796875
Epoch  1, Batch 365 -Loss:  1090.8162 Validation Accuracy: 0.796875
Epoch  1, Batch 366 -Loss:  1892.2716 Validation Accuracy: 0.792969
Epoch  1, Batch 367 -Loss:  1277.6440 Validation Accuracy: 0.792969
Epoch  1, Batch 368 -Loss:  1701.8542 Validation

Epoch  2, Batch  46 -Loss:  1528.9683 Validation Accuracy: 0.812500
Epoch  2, Batch  47 -Loss:   995.7547 Validation Accuracy: 0.812500
Epoch  2, Batch  48 -Loss:  1299.0583 Validation Accuracy: 0.816406
Epoch  2, Batch  49 -Loss:  1187.8984 Validation Accuracy: 0.816406
Epoch  2, Batch  50 -Loss:  1048.8828 Validation Accuracy: 0.812500
Epoch  2, Batch  51 -Loss:  1085.5496 Validation Accuracy: 0.820312
Epoch  2, Batch  52 -Loss:  2056.8787 Validation Accuracy: 0.816406
Epoch  2, Batch  53 -Loss:  1172.9565 Validation Accuracy: 0.820312
Epoch  2, Batch  54 -Loss:  1206.1622 Validation Accuracy: 0.820312
Epoch  2, Batch  55 -Loss:   793.9944 Validation Accuracy: 0.812500
Epoch  2, Batch  56 -Loss:  1154.8826 Validation Accuracy: 0.816406
Epoch  2, Batch  57 -Loss:  2209.2070 Validation Accuracy: 0.816406
Epoch  2, Batch  58 -Loss:  1407.4408 Validation Accuracy: 0.812500
Epoch  2, Batch  59 -Loss:  1094.9517 Validation Accuracy: 0.808594
Epoch  2, Batch  60 -Loss:  1684.2988 Validation

...

Epoch  9, Batch 310 -Loss:   190.1971 Validation Accuracy: 0.875000
Epoch  9, Batch 311 -Loss:   318.4373 Validation Accuracy: 0.875000
Epoch  9, Batch 312 -Loss:   351.0914 Validation Accuracy: 0.875000
Epoch  9, Batch 313 -Loss:   350.0134 Validation Accuracy: 0.875000
Epoch  9, Batch 314 -Loss:   325.3796 Validation Accuracy: 0.875000
Epoch  9, Batch 315 -Loss:   602.2960 Validation Accuracy: 0.875000
Epoch  9, Batch 316 -Loss:   294.6462 Validation Accuracy: 0.875000
Epoch  9, Batch 317 -Loss:   405.4995 Validation Accuracy: 0.878906
Epoch  9, Batch 318 -Loss:   319.9600 Validation Accuracy: 0.878906
Epoch  9, Batch 319 -Loss:   413.9427 Validation Accuracy: 0.875000
Epoch  9, Batch 320 -Loss:   325.0533 Validation Accuracy: 0.878906
Epoch  9, Batch 321 -Loss:   323.5129 Validation Accuracy: 0.878906
Epoch  9, Batch 322 -Loss:   343.1588 Validation Accuracy: 0.878906
Epoch  9, Batch 323 -Loss:   293.3577 Validation Accuracy: 0.875000
Epoch  9, Batch 324 -Loss:   333.6201 Validation

Epoch 10, Batch   2 -Loss:   345.5663 Validation Accuracy: 0.875000
Epoch 10, Batch   3 -Loss:   266.3741 Validation Accuracy: 0.878906
Epoch 10, Batch   4 -Loss:   618.0614 Validation Accuracy: 0.871094
Epoch 10, Batch   5 -Loss:   463.5012 Validation Accuracy: 0.871094
Epoch 10, Batch   6 -Loss:   210.8606 Validation Accuracy: 0.871094
Epoch 10, Batch   7 -Loss:   363.5956 Validation Accuracy: 0.871094
Epoch 10, Batch   8 -Loss:   390.2399 Validation Accuracy: 0.871094
Epoch 10, Batch   9 -Loss:   138.6559 Validation Accuracy: 0.871094
Epoch 10, Batch  10 -Loss:   314.7878 Validation Accuracy: 0.871094
Epoch 10, Batch  11 -Loss:   467.1781 Validation Accuracy: 0.871094
Epoch 10, Batch  12 -Loss:   315.1828 Validation Accuracy: 0.871094
Epoch 10, Batch  13 -Loss:   197.0285 Validation Accuracy: 0.871094
Epoch 10, Batch  14 -Loss:   223.5019 Validation Accuracy: 0.871094
Epoch 10, Batch  15 -Loss:   433.7081 Validation Accuracy: 0.871094
Epoch 10, Batch  16 -Loss:   507.3660 Validation

Epoch 10, Batch 123 -Loss:   392.8456 Validation Accuracy: 0.882812
Epoch 10, Batch 124 -Loss:   431.5576 Validation Accuracy: 0.882812
Epoch 10, Batch 125 -Loss:   185.6795 Validation Accuracy: 0.882812
Epoch 10, Batch 126 -Loss:   293.0289 Validation Accuracy: 0.882812
Epoch 10, Batch 127 -Loss:   293.8749 Validation Accuracy: 0.882812
Epoch 10, Batch 128 -Loss:   336.1841 Validation Accuracy: 0.882812
Epoch 10, Batch 129 -Loss:   299.3408 Validation Accuracy: 0.878906
Epoch 10, Batch 130 -Loss:   166.0758 Validation Accuracy: 0.886719
Epoch 10, Batch 131 -Loss:   318.2360 Validation Accuracy: 0.882812
Epoch 10, Batch 132 -Loss:   322.5070 Validation Accuracy: 0.882812
Epoch 10, Batch 133 -Loss:   209.3455 Validation Accuracy: 0.878906
Epoch 10, Batch 134 -Loss:   258.1602 Validation Accuracy: 0.886719
Epoch 10, Batch 135 -Loss:   345.0686 Validation Accuracy: 0.886719
Epoch 10, Batch 136 -Loss:   311.9088 Validation Accuracy: 0.886719
Epoch 10, Batch 137 -Loss:   319.5907 Validation

Epoch 10, Batch 244 -Loss:   383.6516 Validation Accuracy: 0.886719
Epoch 10, Batch 245 -Loss:   290.0869 Validation Accuracy: 0.882812
Epoch 10, Batch 246 -Loss:   400.0765 Validation Accuracy: 0.886719
Epoch 10, Batch 247 -Loss:   332.3773 Validation Accuracy: 0.886719
Epoch 10, Batch 248 -Loss:   434.0532 Validation Accuracy: 0.886719
Epoch 10, Batch 249 -Loss:   254.8611 Validation Accuracy: 0.886719
Epoch 10, Batch 250 -Loss:   256.8848 Validation Accuracy: 0.878906
Epoch 10, Batch 251 -Loss:   272.0744 Validation Accuracy: 0.878906
Epoch 10, Batch 252 -Loss:   331.8953 Validation Accuracy: 0.882812
Epoch 10, Batch 253 -Loss:   361.8205 Validation Accuracy: 0.878906
Epoch 10, Batch 254 -Loss:   462.8199 Validation Accuracy: 0.878906
Epoch 10, Batch 255 -Loss:   254.6184 Validation Accuracy: 0.878906
Epoch 10, Batch 256 -Loss:   373.6580 Validation Accuracy: 0.878906
Epoch 10, Batch 257 -Loss:   408.1785 Validation Accuracy: 0.875000
Epoch 10, Batch 258 -Loss:   153.7759 Validation

Epoch 10, Batch 365 -Loss:   438.7559 Validation Accuracy: 0.878906
Epoch 10, Batch 366 -Loss:   302.9922 Validation Accuracy: 0.878906
Epoch 10, Batch 367 -Loss:   180.5734 Validation Accuracy: 0.878906
Epoch 10, Batch 368 -Loss:   158.3526 Validation Accuracy: 0.878906
Epoch 10, Batch 369 -Loss:   293.2813 Validation Accuracy: 0.878906
Epoch 10, Batch 370 -Loss:   139.3126 Validation Accuracy: 0.882812
Epoch 10, Batch 371 -Loss:   372.8355 Validation Accuracy: 0.882812
Epoch 10, Batch 372 -Loss:   446.9156 Validation Accuracy: 0.886719
Epoch 10, Batch 373 -Loss:   303.1210 Validation Accuracy: 0.886719
Epoch 10, Batch 374 -Loss:   309.7427 Validation Accuracy: 0.886719
Epoch 10, Batch 375 -Loss:   488.2928 Validation Accuracy: 0.890625
Epoch 10, Batch 376 -Loss:   303.6232 Validation Accuracy: 0.886719
Epoch 10, Batch 377 -Loss:   408.8316 Validation Accuracy: 0.890625
Epoch 10, Batch 378 -Loss:   422.9004 Validation Accuracy: 0.890625
Epoch 10, Batch 379 -Loss:   198.7411 Validation